In [3]:
import networkx as nx
import pandas as pd
from tqdm import tqdm
from IPython.display import display

In [4]:
data_path = "./nodes.dmp"

re_expression = "[|\t]+"

# only read the first three columns
df = pd.read_csv(
    data_path, sep=re_expression, header=None, engine="python", usecols=[0, 1, 2]
)

df.columns = ["Node ID","Parent Node ID", "rank"]

In [5]:
def read_nodes(data_path):
    re_expression = "[|\t]+"

    # only read the first three columns
    df = pd.read_csv(
        data_path, sep=re_expression, header=None, engine="python", usecols=[0, 1, 2]
    )

    df.columns = ["Node ID", "Parent Node ID", "rank"]
    
    return df

df = read_nodes("./nodes.dmp")

In [6]:
df["Node ID"].value_counts().get(1,0)

1

In [7]:
def build(df: pd.DataFrame) -> nx.DiGraph:
    G = nx.DiGraph()
    for _, row in tqdm(df.iterrows()):
        parent_node_id = row['Parent Node ID']
        node_id = row['Node ID']

        G.add_node(parent_node_id)
        G.add_node(node_id)
        
        G.add_edge(parent_node_id, node_id)
    return G

In [8]:
total_G = build(df)

2442791it [04:42, 8641.68it/s] 


In [9]:
print(f"Is the graph a DAG: {nx.is_directed_acyclic_graph(total_G)}") # topological sorting to determine if DAG

Is the graph a DAG: False


In [10]:
def is_rooted(G: nx.DiGraph) -> bool:
    
    # Copy the graphs so that the modifications dont affect the original graph
    G_copy = G.copy()
    
    # Remove self-loop if it exists at the root (node 1)
    if G_copy.has_edge(1, 1):
        G_copy.remove_edge(1, 1)
    
    root_nodes = 0
    for node in G_copy.nodes():
        in_degree = G_copy.in_degree(node)
        if in_degree == 0:
            print(node)
            root_nodes += 1
        elif in_degree > 1:
            print("NCBI taxonomy is not rooted because a node has more than two parents")
            return False
    if root_nodes != 1:
        print(f"NCBI taxonomy is not rooted because it has {root_nodes} roots")
        return False
        
    return True
print(is_rooted(total_G))

1
True


In [11]:
def is_dag(G: nx.DiGraph) -> bool:
    # Copy the graphs so that the modifications dont affect the original graph
    G_copy = G.copy()
    
    # Remove self-loop if it exists at the root (node 1)
    if G_copy.has_edge(1, 1):
        G_copy.remove_edge(1, 1)
    
    is_dag = nx.is_directed_acyclic_graph(G_copy)
    if not is_dag:
        return False
    else:
        print("NCBI taxonomy is DAG")
        return True
    
print(is_dag(total_G))

NCBI taxonomy is DAG
True


there is a cycle from node 1 to it self. It is however the only cylce in the graph, as if i remove this node, the graph becomes acyclic

In [12]:
print(f"There are {len(df['Node ID'])} nodes in the NCBI taxonomy")

There are 2442791 nodes in the NCBI taxonomy


In [13]:
standard_taxonomic_ranks = [
    "kingdom",
    "phylum",
    "class",
    "order",
    "family",
    "genus",
    "species",
]

display(df["rank"].value_counts()[standard_taxonomic_ranks])

rank
kingdom         13
phylum         291
class          461
order         1760
family        9907
genus       104194
species    1995381
Name: count, dtype: int64

In [14]:
filtered_df = df[df['rank'].isin(standard_taxonomic_ranks)]
filtered_G = build(filtered_df)

2112007it [04:30, 7817.30it/s]


## What is the name of the kingdom taxonomic rank in the NCBI taxonomy?

In [15]:
for idx, node in df[df['Parent Node ID'] == 1].iterrows():
    #parent_node_id = node['Parent Node ID']
    #print(node['Parent Node ID'])
    #print(df[df['Node ID'] == parent_node_id]['rank'])

SyntaxError: incomplete input (1570598073.py, line 4)

# Other

In [16]:
df['rank'].value_counts()

rank
species             1995381
no rank              233587
genus                104194
strain                45245
subspecies            27134
family                 9907
varietas               9243
subfamily              3203
tribe                  2304
order                  1760
subgenus               1740
isolate                1322
serotype               1235
clade                   915
superfamily             891
forma specialis         747
forma                   633
subtribe                582
section                 479
class                   461
suborder                373
species group           347
phylum                  291
subclass                166
serogroup               140
infraorder              130
species subgroup        129
superorder               57
subphylum                32
parvorder                26
genotype                 21
subsection               21
infraclass               19
biotype                  17
kingdom                  13
morph          

## How many nodes are there in the NCBI taxonomy, once restricted to the seven standard taxonomic ranks?

In [17]:
print(f"There are {len(filtered_df)} nodes in the restricted NCBI taxonomy")

There are 2112007 nodes in the restricted NCBI taxonomy
